In [1]:
%%writefile matmul.cu
#include <cuda_runtime.h>
#include <stdio.h>

// Kernel 定义
__global__ void matrix_multiplication_kernel(const float* A, const float* B, float* C, int M, int N, int K) {
    int row =  blockDim.y * blockIdx.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    
    if(col >= K || row >= M){
        return;
    }
    
    float acc = 0.0f;
    for(int i = 0; i < N; i++){
        acc += A[row * N + i] * B[i * K + col];
    }
    C[row * K + col] = acc; 
}

// 宿主端 wrapper 函数
extern "C" void solve(const float* A, const float* B, float* C, int M, int N, int K) {
    dim3 threadsPerBlock(16, 16);
    // 注意：grid 的计算需要向上取整，你的代码已经包含了这个逻辑，但建议加上括号保证运算顺序
    dim3 blocksPerGrid((K + threadsPerBlock.x - 1) / threadsPerBlock.x,
                       (M + threadsPerBlock.y - 1) / threadsPerBlock.y);
    
    matrix_multiplication_kernel<<<blocksPerGrid, threadsPerBlock>>>(A, B, C, M, N, K);
    
    // 检查是否有错误发生（调试用）
    cudaError_t err = cudaGetLastError();
    if (err != cudaSuccess) {
        printf("CUDA Error: %s\n", cudaGetErrorString(err));
    }
    
    // 等待 GPU 完成
    cudaDeviceSynchronize();
}

Overwriting matmul.cu


上面的实现有一个很大的问题就是，在每一个线程里面，为了计算C里面的一个位置上面的值，需要去全局内存上访问col次数据A，也需要访问col次数据B，导致数据等待时间大大增加。解决方法在于，使用线程块里面的共享内存，在一个线程块里面，先读取A和B的一部分数据到线程块里面的共享内存里面，然后进行计算，接着读取下一部分。通过分块避免所有线程去读取全局内存里面的数据。

In [2]:
%%writefile matmul_v2.cu
#include <stdio.h>
#include <cuda_runtime.h>
#define TILE_WIDTH 32 // 假设 BlockDim 为 32x32

__global__ void matrix_multiplication_shared_mem(const float* __restrict__ A, const float* __restrict__ B, float* C, int M, int N, int K) {
    // 申请共享内存
    __shared__ float As[TILE_WIDTH][TILE_WIDTH];
    __shared__ float Bs[TILE_WIDTH][TILE_WIDTH];

    int bx = blockIdx.x;
    int by = blockIdx.y;
    int tx = threadIdx.x;
    int ty = threadIdx.y;

    // 计算当前线程在全局矩阵 C 中负责的坐标
    int row = by * TILE_WIDTH + ty;
    int col = bx * TILE_WIDTH + tx;

    float acc = 0.0f;

    // 循环遍历所有的 Tile (以 TILE_WIDTH 为步长遍历 N 维度)
    for (int t = 0; t < (N + TILE_WIDTH - 1) / TILE_WIDTH; ++t) {

        // 1. 协作加载 A 的 Tile 到共享内存
        // 边界检查：防止索引越界 (Padding 0)
        if (row < M && t * TILE_WIDTH + tx < N) {
            As[ty][tx] = A[row * N + t * TILE_WIDTH + tx];
        } else {
            As[ty][tx] = 0.0f;
        }

        // 2. 协作加载 B 的 Tile 到共享内存
        if (col < K && t * TILE_WIDTH + ty < N) {
            // 注意这里 B 的索引：行是 t*TILE_WIDTH + ty, 列是 col
            Bs[ty][tx] = B[(t * TILE_WIDTH + ty) * K + col];
        } else {
            Bs[ty][tx] = 0.0f;
        }

        // 3. 必须同步！确保所有线程都加载完了数据
        __syncthreads();

        // 4. 在共享内存上进行计算
        for (int i = 0; i < TILE_WIDTH; ++i) {
            acc += As[ty][i] * Bs[i][tx];
        }

        // 5. 必须同步！确保在加载下一个 Tile 之前，当前 Tile 的数据已经用完
        __syncthreads();
    }

    // 写回结果
    if (row < M && col < K) {
        C[row * K + col] = acc;
    }
}

// 宿主端 wrapper 函数
extern "C" void solve(const float* A, const float* B, float* C, int M, int N, int K) {
    dim3 threadsPerBlock(TILE_WIDTH, TILE_WIDTH);
    // 注意：grid 的计算需要向上取整，你的代码已经包含了这个逻辑，但建议加上括号保证运算顺序
    dim3 blocksPerGrid((K + threadsPerBlock.x - 1) / threadsPerBlock.x,
                       (M + threadsPerBlock.y - 1) / threadsPerBlock.y);
    
    matrix_multiplication_shared_mem<<<blocksPerGrid, threadsPerBlock>>>(A, B, C, M, N, K);
    
    // 检查是否有错误发生（调试用）
    cudaError_t err = cudaGetLastError();
    if (err != cudaSuccess) {
        printf("CUDA Error: %s\n", cudaGetErrorString(err));
    }
    
    // 等待 GPU 完成
    cudaDeviceSynchronize();
}

Overwriting matmul_v2.cu


In [3]:
%%writefile matmul_v3.cu
#include <stdio.h>
#include <cuda_runtime.h>
#define TILE_WIDTH 32 // 假设 BlockDim 为 32x32

__global__ void matrix_multiplication_shared_mem(const float *__restrict__ A, const float *__restrict__ B, float *C, int M, int N, int K)
{
    // 申请共享内存
    __shared__ float As[TILE_WIDTH][TILE_WIDTH];
    __shared__ float Bs[TILE_WIDTH][TILE_WIDTH+1];

    int ty = threadIdx.y;
    int tx = threadIdx.x;

    int row = blockIdx.y * TILE_WIDTH + ty;
    int col = blockIdx.x * TILE_WIDTH + tx;

    float acc = 0.0f;

    for (int i = 0; i < (N + TILE_WIDTH - 1) / TILE_WIDTH; i++)
    {
        // 读取数据
        if (row < M && TILE_WIDTH * i + tx < N)
        {
            As[ty][tx] = A[row * N + TILE_WIDTH * i + tx];
        }
        else
        {
            As[ty][tx] = 0.0f;
        }
        if (col < K && TILE_WIDTH * i + ty < N)
        {
            Bs[ty][tx] = B[(TILE_WIDTH * i + ty) * K + col];
        }
        else
        {
            Bs[ty][tx] = 0.0f;
        }
        __syncthreads(); // 等待线程块里面的线程都搬运完成

        for (int j = 0; j < TILE_WIDTH; j++)
        {
            acc += As[ty][j] * Bs[j][tx]; //访问线程块里面的共享内存时，没有合并访问，只在乎bank config
        }

        __syncthreads();
    }
    if (row<M && col<K)
    {
        C[row*K+col] = acc;
    }
}

// 宿主端 wrapper 函数
extern "C" void solve(const float *A, const float *B, float *C, int M, int N, int K)
{
    dim3 threadsPerBlock(TILE_WIDTH, TILE_WIDTH);
    // 注意：grid 的计算需要向上取整，你的代码已经包含了这个逻辑，但建议加上括号保证运算顺序
    dim3 blocksPerGrid((K + threadsPerBlock.x - 1) / threadsPerBlock.x,
                       (M + threadsPerBlock.y - 1) / threadsPerBlock.y);

    matrix_multiplication_shared_mem<<<blocksPerGrid, threadsPerBlock>>>(A, B, C, M, N, K);

    // 检查是否有错误发生（调试用）
    cudaError_t err = cudaGetLastError();
    if (err != cudaSuccess)
    {
        printf("CUDA Error: %s\n", cudaGetErrorString(err));
    }

    // 等待 GPU 完成
    cudaDeviceSynchronize();
}


Overwriting matmul_v3.cu


In [ ]:
%%writefile matmul_v4.cu
#include <stdio.h>
#include <cuda_runtime.h>

// 宏定义块大小
// TS (Tile Size): 每个 Block 计算 64x64 的 C
// WPT (Work Per Thread): 每个线程计算 4x4 的 C
// TS_K: K 维度(你的代码里是 N 维度)的分块大小，设为 8 或 16
#define TS 64
#define WPT 4
#define TS_K 16 

// 优化后的 Kernel
__global__ void matrix_multiplication_optimized(
    const float* __restrict__ A, 
    const float* __restrict__ B, 
    float* __restrict__ C, 
    int M, int N, int K) 
{
    // 每个 Block 处理 C 中 TS x TS (64x64) 的区域
    // 线程块维度: dim3(TS/WPT, TS/WPT) -> (16, 16) -> 256 个线程
    
    // 1. 声明共享内存
    // As: 存储 A 的切片 [TS][TS_K] -> [64][16]
    // Bs: 存储 B 的切片 [TS_K][TS] -> [16][64]
    __shared__ float As[TS][TS_K];
    __shared__ float Bs[TS_K][TS];

    // 2. 声明寄存器
    // accum: 累加器，每个线程负责计算 4x4 = 16 个元素
    float accum[WPT][WPT] = {0.0f};
    
    // reg_A, reg_B: 用于在内循环中缓存从 SMEM 读取的值
    float reg_A[WPT];
    float reg_B[WPT];

    // 线程 ID 和 Block ID
    int tx = threadIdx.x; // range 0-15
    int ty = threadIdx.y; // range 0-15
    int bx = blockIdx.x;
    int by = blockIdx.y;

    // 当前线程负责的 C 矩阵起始坐标 (C 的分块左上角 + 线程偏移)
    // 每个线程覆盖 WPT(4) 个像素宽/高
    int row_c = by * TS + ty * WPT; 
    int col_c = bx * TS + tx * WPT;

    // 3. 循环遍历 N 维度 (步长 TS_K = 16)
    for (int t = 0; t < N; t += TS_K) {
        
        // --- 加载数据到 Shared Memory (协作加载) ---
        // 我们有 256 个线程。
        // 需要加载 A 的 Tile: 64行 * 16列 = 1024 元素。每个线程加载 1024/256 = 4 个元素。
        // 需要加载 B 的 Tile: 16行 * 64列 = 1024 元素。每个线程加载 4 个元素。
        
        // 加载 As (A 的子块): 
        // 这里的逻辑是将 256 个线程映射到 64x16 的区域
        // 我们使用 float4 向量化加载来极致优化带宽
        
        // 计算当前线程加载 As 的位置
        // 将 16x16 的线程块视为 256 个线性线程
        int tid = ty * (TS / WPT) + tx; // 0 ~ 255
        
        // 映射到 As[64][16]: 每一行 16 个元素，如果是 float4 就是 4 个 float4
        // 256 个线程，每个加载 1 个 float4 (4个float)，正好 1024 个 float
        // As 的行索引
        int load_a_row = tid / (TS_K / 4); 
        int load_a_col = (tid % (TS_K / 4)) * 4;
        
        // 从全局内存 A 加载到 As
        // 全局索引: A[(by * TS + load_a_row) * N + (t + load_a_col)]
        // 注意边界检查省略了，假设维度对其
        if (by * TS + load_a_row < M && t + load_a_col < N) {
             // 使用 float4 指针强转进行向量加载
             float4 tmp = reinterpret_cast<const float4*>(&A[(by * TS + load_a_row) * N + (t + load_a_col)])[0];
             As[load_a_row][load_a_col + 0] = tmp.x;
             As[load_a_row][load_a_col + 1] = tmp.y;
             As[load_a_row][load_a_col + 2] = tmp.z;
             As[load_a_row][load_a_col + 3] = tmp.w;
        }

        // 加载 Bs (B 的子块): [16][64]
        // 同样用 tid 映射。每行 64 个元素 = 16 个 float4。
        // 总共 16 行。总 float4 数 = 16 * 16 = 256。正好每个线程取 1 个 float4。
        int load_b_row = tid / (TS / 4);
        int load_b_col = (tid % (TS / 4)) * 4;

        if (t + load_b_row < N && bx * TS + load_b_col < K) {
             float4 tmp = reinterpret_cast<const float4*>(&B[(t + load_b_row) * K + (bx * TS + load_b_col)])[0];
             Bs[load_b_row][load_b_col + 0] = tmp.x;
             Bs[load_b_row][load_b_col + 1] = tmp.y;
             Bs[load_b_row][load_b_col + 2] = tmp.z;
             Bs[load_b_row][load_b_col + 3] = tmp.w;
        }

        __syncthreads(); // 等待数据加载完成

        // --- 在寄存器上进行计算 ---
        // 遍历 Shared Memory 中的 TS_K (16) 维度
        #pragma unroll
        for (int k = 0; k < TS_K; ++k) {
            
            // 1. 将所需的 As 和 Bs 数据预加载到寄存器
            // 每个线程计算 4x4，需要 As 的一列 4 个值，Bs 的一行 4 个值
            for (int i = 0; i < WPT; ++i) {
                reg_A[i] = As[ty * WPT + i][k];
                reg_B[i] = Bs[k][tx * WPT + i];
            }

            // 2. 外积计算 (Outer Product)
            // 计算 4x4 的结果，复用 reg_A 和 reg_B
            for (int row = 0; row < WPT; ++row) {
                for (int col = 0; col < WPT; ++col) {
                    accum[row][col] += reg_A[row] * reg_B[col];
                }
            }
        }
        
        __syncthreads(); // 等待计算完成，准备加载下一块
    }

    // 4. 写回结果到全局内存
    // 每个线程写回 4x4 个点
    for (int row = 0; row < WPT; ++row) {
        for (int col = 0; col < WPT; ++col) {
            int global_row = row_c + row;
            int global_col = col_c + col;
            
            if (global_row < M && global_col < K) {
                C[global_row * K + global_col] = accum[row][col];
            }
        }
    }
}

// Host 端调用示例
extern "C" void solve(const float* d_A, const float* d_B, float* d_C, int M, int N, int K) {
    // 线程块大小: 16x16 = 256 线程
    dim3 threadsPerBlock(TS / WPT, TS / WPT); 
    
    // Grid 大小: 因为每个 Block 处理 64x64，所以除以 TS(64)
    dim3 numBlocks((K + TS - 1) / TS, (M + TS - 1) / TS);

    matrix_multiplication_optimized<<<numBlocks, threadsPerBlock>>>(d_A, d_B, d_C, M, N, K);
}

Overwriting matmul_v4.cu


In [5]:
import os
import subprocess
for v in os.listdir(os.path.abspath('.')):
    prefix, end = os.path.splitext(v)
    if end == '.cu':
        subprocess.run(f"nvcc -shared -o {prefix}.so {prefix}.cu -Xcompiler -fPIC", shell=True)
print(os.listdir())

['.virtual_documents', 'matmul_v2.cu', 'matmul.so', 'transpose.cu', 'transpose.so', 'matmul_v3.so', 'matmul_v4.cu', 'matmul_v4.so', 'matmul_v3.cu', 'matmul.cu', 'matmul_v2.so']


In [6]:
import torch
import ctypes
import numpy as np
import time

# --- 测试部分 ---

# 设置维度
M, N, K = 1024, 1024, 1024

print(f"正在进行矩阵乘法测试: [{M}x{N}] * [{N}x{K}]")

# 创建随机数据 (在 GPU 上)
device = torch.device("cuda")
A = torch.randn(M, N, device=device, dtype=torch.float32)
B = torch.randn(N, K, device=device, dtype=torch.float32)

# 2. 运行 PyTorch 内置矩阵乘法 (作为标准答案)
C_torch = torch.matmul(A, B)

print("Pytorch: ")
%timeit torch.matmul(A, B); torch.cuda.synchronize()
print()

for v in sorted(os.listdir('.')):
    prefix, end = os.path.splitext(v)
    
    if end != '.so':
        continue

    # 1. 加载编译好的 .so 库
    lib = ctypes.CDLL(f'./{v}')

    # 2. 定义函数参数类型
    # void solve(const float* A, const float* B, float* C, int M, int N, int K)
    # 指针对应 c_void_p (因为我们要传显存地址), int 对应 c_int
    lib.solve.argtypes = [
        ctypes.c_void_p, 
        ctypes.c_void_p, 
        ctypes.c_void_p, 
        ctypes.c_int, 
        ctypes.c_int, 
        ctypes.c_int
    ]

    def cuda_matmul(a_tensor, b_tensor):
        # 获取维度
        M, N = a_tensor.shape
        N_b, K = b_tensor.shape
        
        assert N == N_b, f"矩阵维度不匹配: {N} != {N_b}"
        
        # 初始化输出矩阵 C (在 GPU 上分配)
        c_tensor = torch.zeros((M, K), device='cuda', dtype=torch.float32)
        
        # 确保输入是连续内存且在 GPU 上
        if not a_tensor.is_contiguous(): a_tensor = a_tensor.contiguous()
        if not b_tensor.is_contiguous(): b_tensor = b_tensor.contiguous()
        
        # 3. 调用 CUDA 函数
        # 注意：必须传入 data_ptr()，这是物理显存地址
        lib.solve(
            ctypes.c_void_p(a_tensor.data_ptr()),
            ctypes.c_void_p(b_tensor.data_ptr()),
            ctypes.c_void_p(c_tensor.data_ptr()),
            ctypes.c_int(M),
            ctypes.c_int(N),
            ctypes.c_int(K)
        )
        
        return c_tensor

    # 1. 运行你的 CUDA Kernel
    C_custom = cuda_matmul(A, B)
    print(f"{v}")
    # 3. 验证结果
    # 允许一点浮点误差
    if torch.allclose(C_custom, C_torch, atol=1e-3):
        print(f"✅ 测试通过！结果正确。")
        %timeit cuda_matmul(A, B); torch.cuda.synchronize()
    else:
        print("❌ 测试失败。结果不一致。")
        print("最大误差:", (C_custom - C_torch).abs().max().item())
    print()

正在进行矩阵乘法测试: [1024x1024] * [1024x1024]
Pytorch: 
653 µs ± 4.85 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)

matmul.so
✅ 测试通过！结果正确。
5.02 ms ± 14.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

matmul_v2.so
✅ 测试通过！结果正确。
2.61 ms ± 11.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

matmul_v3.so
✅ 测试通过！结果正确。
2.65 ms ± 7.73 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

matmul_v4.so
✅ 测试通过！结果正确。
834 µs ± 9.72 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)

transpose.so
❌ 测试失败。结果不一致。
最大误差: 176.31954956054688

